Pos Tech - MLET - Tech Challenge - Fase 4

Aqui está a estrutura para o Tech Challenge, cobrindo todas as etapas desde a coleta de dados até o deploy da API. A implementação será detalhada e ao final haverá um pipeline funcional.

1. Coleta e Pré-processamento dos Dados

Utilizaremos a biblioteca yfinance para obter os dados históricos de preços de ações. Escolhemos a ação da Microsoft (MSFT) como exemplo.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Configuração do período e símbolo da ação
symbol = 'MSFT'
start_date = '2018-01-01'
end_date = '2024-07-20'

# Coleta de dados
df = yf.download(symbol, start=start_date, end=end_date)

# Pré-processamento: Seleção da coluna de fechamento e normalização
data = df[['Close']].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Criação de janelas para LSTM
def create_dataset(dataset, look_back=60):
    X, y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:i+look_back, 0])
        y.append(dataset[i+look_back, 0])
    return np.array(X), np.array(y)

look_back = 60
X, y = create_dataset(scaled_data, look_back)
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape para entrada LSTM

[*********************100%%**********************]  1 of 1 completed


2. Desenvolvimento do Modelo LSTM

Usaremos TensorFlow para criar e treinar o modelo LSTM.

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Construção do modelo
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

# Compilação e treinamento
model.compile(optimizer='adam', loss='mean_squared_error')

# Divisão dos dados em treino e teste
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

# Avaliação
loss = model.evaluate(X_test, y_test)
print(f'Model Loss: {loss}')

c:\Users\lucia\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - loss: 0.0332 - val_loss: 0.0068
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0017 - val_loss: 9.4379e-04
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.0018 - val_loss: 0.0014
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0014 - val_loss: 6.5929e-04
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0012 - val_loss: 0.0038
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 9.4743e-04 - val_loss: 0.0020
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0010 - val_loss: 0.0029
Epoch 9/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 9.9782e-04 - val_loss: 0.0019
Epoch 10/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 8.7546e-04 - val_loss: 0.0014
Epoch 11/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 8.8979e-04 - val_loss: 7.7746e-04
Epoch 12/50
40/40 ━━━━━━━━━━━━━━━━━

3. Salvamento e Exportação do Modelo

Salvaremos o modelo em formato .h5.

In [3]:
model.save(r'C:\Users\lucia\OneDrive\Área de Trabalho\lstm_stock_model.h5')

4. Deploy do Modelo com FastAPI

A API será criada com o framework FastAPI e integrará o modelo salvo.

In [4]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import tensorflow as tf

# Carregar o modelo
model = tf.keras.models.load_model('lstm_stock_model.h5')

# Configuração da API
app = FastAPI()

class PredictionRequest(BaseModel):
    data: list

@app.post("/predict/")
def predict_stock_price(request: PredictionRequest):
    # Receber dados e fazer predição
    input_data = np.array(request.data).reshape(1, -1, 1)
    prediction = model.predict(input_data)
    return {"prediction": float(prediction[0][0])}

5. Escalabilidade e Monitoramento

Docker: Criar um contêiner para o deploy.

In [6]:
Em linguagem dockerfile

FROM python:3.9-slim

WORKDIR /app

COPY . .

RUN pip install fastapi uvicorn tensorflow

CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]

SyntaxError: invalid syntax (3468116675.py, line 1)

Monitoramento:
Uso de ferramentas como Prometheus e Grafana para rastrear métricas de tempo de resposta e uso de recursos.

Entregáveis

Repositório GitHub com:

Código-fonte do modelo LSTM.

Script de deploy com Docker.

Link para a API em produção (usando AWS, GCP ou outra plataforma).

Vídeo demonstrativo explicando a funcionalidade da API.